In [1]:
# 构建计算图——LSTM模型
#      embedding
#      LSTM
#      fc
#      train_op
# 训练流程代码
# 数据集封装
#      api: next_batch(batch_size)
# 词表封装
#      api: sentence2id(text_sentence): 句子转换id
# 类别封装
#      api: category2id(text_category)

import tensorflow as tf
import os
import sys
import numpy as np
import math

tf.logging.set_verbosity(tf.logging.INFO)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
def get_default_params():
    return tf.contrib.training.HParams(
        num_embedding_size = 16,  
        num_timesteps = 50, # minibatch长度对齐
        num_filters = 128,
        num_kernel_size = 3,
        num_fc_nodes = 32,
        batch_size = 100,
        learning_rate = 0.001,
        num_word_threshold = 10  # 词频阈值
    )

hps = get_default_params()

# input file
train_file = './data/cnews.train.seg.txt'
val_file = './data/cnews.val.seg.txt'
test_file = './data/cnews.test.seg.txt'
vocab_file = './data/cnews.vocab.txt'
category_file = './data/cnews.category.txt'
output_file = './data/run_text_rnn'

if not os.path.exists(output_file):
    os.mkdir(output_file)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
print(hps.num_word_threshold)

10


In [4]:
class Vocab:
    def __init__(self, filename, num_word_threshold):
        self._word_to_id = {}
        self._unk = -1
        self._num_word_threshold = num_word_threshold
        self._read_dict(filename)
    
    def _read_dict(self, filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
        for line in lines:
            word, frequency = line.strip('\r\n').split('\t')
            frequency = int(frequency)
            if frequency < self._num_word_threshold:
                continue
            idx = len(self._word_to_id) # id递增
            if word == '<UNK>':
                self._unk = idx
            self._word_to_id[word] = idx
    
    def word_to_id(self, word):
        return self._word_to_id.get(word, self._unk)
    
    @property
    def unk(self):
        return self._unk
    
    def size(self):
        return len(self._word_to_id)
        
    def sentence_to_id(self, sentence):
        word_ids = [self.word_to_id(cur_word) for cur_word in sentence.split()]
        return word_ids
    
    
class CategoryDict:
    def __init__(self, filename):
        self._category_to_id = {}
        with open(filename, 'r') as f:
            lines = f.readlines()
        for line in lines:
            category = line.strip('\r\n')
            idx = len(self._category_to_id)
            self._category_to_id[category] = idx
    
    def size(self):
        return len(self._category_to_id)
        
    def category_to_id(self, category):
        if not category in self._category_to_id:
            raise Execption("%s is not in our category list" % category_name);
        return self._category_to_id[category]
        
        
vocab = Vocab(vocab_file, hps.num_word_threshold)
vocab_size = vocab.size()
tf.logging.info('vocab_size: %d' % vocab.size())

# test_str = '的 在 了 是'
# print(vocab.sentence_to_id(test_str))

category_vocab = CategoryDict(category_file)
num_classes = category_vocab.size()
tf.logging.info('num_classes: %d' % num_classes)
test_str = '时尚'
tf.logging.info('label: %s, id: %d' % (test_str, category_vocab.category_to_id(test_str)))

INFO:tensorflow:vocab_size: 77323
INFO:tensorflow:num_classes: 10
INFO:tensorflow:label: 时尚, id: 5


In [5]:
class TextDataset:
    def __init__(self, filename, vocab, category_vocab, num_timesteps):
        self._vocab = vocab
        self._category_vocab = category_vocab
        self._num_timesteps = num_timesteps
        # matrix
        self._inputs = []
        # vector
        self._outputs = []
        self._indicator = 0
        self._parse_file(filename)
        
    def _parse_file(self, filename):
        tf.logging.info('Loading data from %s', filename)
        with open(filename, 'r') as f:
            lines = f.readlines()
        for line in lines:
            label, content = line.strip('\r\n').split('\t')
            id_label = self._category_vocab.category_to_id(label)
            id_words = self._vocab.sentence_to_id(content)
            id_words = id_words[0: self._num_timesteps] # 长的部分做截断
            padding_num = self._num_timesteps - len(id_words)
            id_words = id_words + [self._vocab.unk for i in range(padding_num)]
            self._inputs.append(id_words)
            self._outputs.append(id_label)
        self._inputs = np.asarray(self._inputs, dtype=np.int32)
        self._outputs = np.asarray(self._outputs, dtype=np.int32)
        self._random_shuffle()
        
    def _random_shuffle(self):
        p = np.random.permutation(len(self._inputs))
        self._inputs = self._inputs[p]
        self._outputs = self._outputs[p]
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > len(self._inputs):
            self._random_shuffle()
            self._indicator = 0
            end_indicator = batch_size
        if end_indicator > len(self._inputs):
            raise Execption("batch_size: %d is too large" % batch_size)
        
        batch_inputs = self._inputs[self._indicator: end_indicator]
        batch_outputs = self._outputs[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_inputs, batch_outputs
    
train_dataset = TextDataset(train_file, vocab, category_vocab, hps.num_timesteps)
val_dataset = TextDataset(val_file, vocab, category_vocab, hps.num_timesteps)
test_dataset = TextDataset(test_file, vocab, category_vocab, hps.num_timesteps)

print(train_dataset.next_batch(2))
print(val_dataset.next_batch(2))
print(test_dataset.next_batch(2))

INFO:tensorflow:Loading data from ./data/cnews.train.seg.txt
INFO:tensorflow:Loading data from ./data/cnews.val.seg.txt
INFO:tensorflow:Loading data from ./data/cnews.test.seg.txt
(array([[75924,  6833,  2219,  2580,  2324,  1273,  6685,     0,    23,
          503,    44,    30,   174,    23,   294,    44,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [11498,    11,    47,   305,  6399,   745, 30020,   672, 25884,
         1550,   180,    23,   595,   814,  5609,  9712,     5,     0,
         4512, 11498,     4,   258,  1653,     2,   348, 20479,   127,
          866,    29,   447,     1,    47,     4,  8503,  4846,     5,
        57845,    29, 17855,     1, 29660,  1328, 54042,    10, 30020,
          672,     1,   305,  6399,    85]], dtype=int32), array(

In [6]:
def create_model(hps, vocab_size, num_classes):
    num_timesteps = hps.num_timesteps
    batch_size = hps.batch_size
    
    inputs = tf.placeholder(tf.int32, (batch_size, num_timesteps))
    outputs = tf.placeholder(tf.int32, (batch_size, ))
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    global_step = tf.Variable(tf.zeros([], tf.int64), name='global_step', trainable=False)
    
    embedding_initializer = tf.random_uniform_initializer(-1.0, 1.0)
    with tf.variable_scope('embedding', initializer=embedding_initializer):
        embeddings = tf.get_variable(
            'embedding',
            [vocab_size, hps.num_embedding_size],
            tf.float32)
        # [1, 10, 7] -> [embeddings[1], embeddings[10], embeddings[7]]
        embed_inputs = tf.nn.embedding_lookup(embeddings, inputs)
        
    scale = 1.0 / math.sqrt(hps.num_embedding_size + hps.num_filters) / 3.0
    cnn_init = tf.random_uniform_initializer(-scale, scale)
    with tf.variable_scope('cnn', initializer = cnn_init):
        # embed_inputs: [batch_size, timesteps, embed_size]
        # conv1d: [batch_size, timesteps, num_filters]
        conv1d = tf.layers.conv1d(
            embed_inputs,
            hps.num_filters,
            hps.num_kernel_size,
            activation = tf.nn.relu,
        )
        global_maxpooling = tf.reduce_max(conv1d, axis=[1])
        
    
    """  
    scale = 1.0 / math.sqrt(hps.num_embedding_size + hps.num_lstm_nodes[-1]) / 3.0
    lstm_init = tf.random_uniform_initializer(-scale, scale)
    with tf.variable_scope('lstm_nn', initializer = lstm_init):
        cells = []
        for i in range(hps.num_lstm_layers):
            cell = tf.contrib.rnn.BasicLSTMCell(
                hps.num_lstm_nodes[i],
                state_is_tuple = True)
            cell = tf.contrib.rnn.DropoutWrapper(
                cell,
                output_keep_prob = keep_prob)
            cells.append(cell)
        cell = tf.contrib.rnn.MultiRNNCell(cells)
        
        initial_state = cell.zero_state(batch_size, tf.float32)
        # rnn_outputs: [batch_size, num_timesteps, lstm_outputs[-1]]
        rnn_outputs, _ = tf.nn.dynamic_rnn(cell, embed_inputs, initial_state = initial_state)
        last = rnn_outputs[:, -1, :]
    """
    
    
    fc_init = tf.uniform_unit_scaling_initializer(factor=1.0)
    with tf.variable_scope('fc', initializer=fc_init):
        fc1 = tf.layers.dense(global_maxpooling, hps.num_fc_nodes, activation=tf.nn.relu, name='fc1')
        fc1_dropout = tf.contrib.layers.dropout(fc1, keep_prob)
        logits = tf.layers.dense(fc1_dropout,num_classes,name='fc2')
        
    with tf.name_scope('metrics'):
        softmax_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=outputs)
        loss = tf.reduce_mean(softmax_loss)
        # [0, 1, 5, 4, 2] -> argmax: 2
        y_pred = tf.argmax(tf.nn.softmax(logits), 1, output_type=tf.int32)
        correct_pred = tf.equal(outputs, y_pred)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    with tf.name_scope('train_op'):
        train_op = tf.train.AdamOptimizer(hps.learning_rate).minimize(loss, global_step=global_step)
        """
        tvars = tf.trainable_variables()
        for var in tvars:
            tf.logging.info('variable name: %s' %(var.name))
        grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), hps.clip_lstm_grads)
        optimizer = tf.train.AdamOptimizer(hps.learning_rate)
        train_op = optimizer.apply_gradients(zip(grads, tvars), global_step = global_step)
        """
        
    return ((inputs, outputs, keep_prob), (loss, accuracy), (train_op, global_step))

placeholders, metrics, others = create_model(hps, vocab_size, num_classes)
inputs, outputs, keep_prob = placeholders
loss, accuracy = metrics
train_op, global_step = others

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv1d instead.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [7]:
init_op = tf.global_variables_initializer()
train_keep_prob_value = 0.8
test_keep_prob_value = 1.0

num_train_steps = 10000

# Train: 100%
# Valid: 95.7%
# Test:  95.2%
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(num_train_steps):
        if i % 150 ==0:
            batch_inputs,batch_labels = test_dataset.next_batch(hps.batch_size)
        else:
            batch_inputs, batch_labels = train_dataset.next_batch(hps.batch_size)
        outputs_val = sess.run([loss, accuracy, train_op, global_step], 
        feed_dict={
            inputs: batch_inputs,
            outputs: batch_labels,
            keep_prob: train_keep_prob_value,
        })
        loss_val, accuracy_val, _, global_step_val = outputs_val
        if global_step_val % 150 == 0:
            tf.logging.info("teststep: %5d,loss: %3.3f,accuracy:%3.5f" % (global_step_val,loss_val,accuracy_val))
        if global_step_val % 20 == 0:
            tf.logging.info("trainstep: %5d,loss: %3.3f,accuracy:%3.5f" % (global_step_val,loss_val,accuracy_val))

INFO:tensorflow:trainstep:    20,loss: 2.283,accuracy:0.15000
INFO:tensorflow:trainstep:    40,loss: 2.215,accuracy:0.23000
INFO:tensorflow:trainstep:    60,loss: 2.108,accuracy:0.25000
INFO:tensorflow:trainstep:    80,loss: 1.872,accuracy:0.48000
INFO:tensorflow:trainstep:   100,loss: 1.830,accuracy:0.40000
INFO:tensorflow:trainstep:   120,loss: 1.524,accuracy:0.55000
INFO:tensorflow:trainstep:   140,loss: 1.426,accuracy:0.50000
INFO:tensorflow:teststep:   150,loss: 1.201,accuracy:0.63000
INFO:tensorflow:trainstep:   160,loss: 1.302,accuracy:0.57000
INFO:tensorflow:trainstep:   180,loss: 1.181,accuracy:0.65000
INFO:tensorflow:trainstep:   200,loss: 1.211,accuracy:0.57000
INFO:tensorflow:trainstep:   220,loss: 1.364,accuracy:0.59000
INFO:tensorflow:trainstep:   240,loss: 1.150,accuracy:0.64000
INFO:tensorflow:trainstep:   260,loss: 1.117,accuracy:0.61000
INFO:tensorflow:trainstep:   280,loss: 0.931,accuracy:0.74000
INFO:tensorflow:teststep:   300,loss: 0.987,accuracy:0.71000
INFO:tenso

INFO:tensorflow:trainstep:  2380,loss: 0.179,accuracy:0.94000
INFO:tensorflow:teststep:  2400,loss: 0.181,accuracy:0.95000
INFO:tensorflow:trainstep:  2400,loss: 0.181,accuracy:0.95000
INFO:tensorflow:trainstep:  2420,loss: 0.351,accuracy:0.90000
INFO:tensorflow:trainstep:  2440,loss: 0.178,accuracy:0.95000
INFO:tensorflow:trainstep:  2460,loss: 0.168,accuracy:0.95000
INFO:tensorflow:trainstep:  2480,loss: 0.123,accuracy:0.97000
INFO:tensorflow:trainstep:  2500,loss: 0.201,accuracy:0.93000
INFO:tensorflow:trainstep:  2520,loss: 0.125,accuracy:0.96000
INFO:tensorflow:trainstep:  2540,loss: 0.117,accuracy:0.98000
INFO:tensorflow:teststep:  2550,loss: 0.262,accuracy:0.92000
INFO:tensorflow:trainstep:  2560,loss: 0.184,accuracy:0.96000
INFO:tensorflow:trainstep:  2580,loss: 0.230,accuracy:0.95000
INFO:tensorflow:trainstep:  2600,loss: 0.237,accuracy:0.95000
INFO:tensorflow:trainstep:  2620,loss: 0.114,accuracy:0.96000
INFO:tensorflow:trainstep:  2640,loss: 0.196,accuracy:0.94000
INFO:tenso

INFO:tensorflow:trainstep:  4720,loss: 0.040,accuracy:0.99000
INFO:tensorflow:trainstep:  4740,loss: 0.021,accuracy:1.00000
INFO:tensorflow:trainstep:  4760,loss: 0.016,accuracy:1.00000
INFO:tensorflow:trainstep:  4780,loss: 0.030,accuracy:1.00000
INFO:tensorflow:teststep:  4800,loss: 0.021,accuracy:0.99000
INFO:tensorflow:trainstep:  4800,loss: 0.021,accuracy:0.99000
INFO:tensorflow:trainstep:  4820,loss: 0.069,accuracy:0.97000
INFO:tensorflow:trainstep:  4840,loss: 0.207,accuracy:0.97000
INFO:tensorflow:trainstep:  4860,loss: 0.045,accuracy:0.99000
INFO:tensorflow:trainstep:  4880,loss: 0.020,accuracy:1.00000
INFO:tensorflow:trainstep:  4900,loss: 0.019,accuracy:1.00000
INFO:tensorflow:trainstep:  4920,loss: 0.019,accuracy:1.00000
INFO:tensorflow:trainstep:  4940,loss: 0.035,accuracy:0.99000
INFO:tensorflow:teststep:  4950,loss: 0.031,accuracy:0.99000
INFO:tensorflow:trainstep:  4960,loss: 0.031,accuracy:1.00000
INFO:tensorflow:trainstep:  4980,loss: 0.134,accuracy:0.96000
INFO:tenso

INFO:tensorflow:trainstep:  7060,loss: 0.008,accuracy:1.00000
INFO:tensorflow:trainstep:  7080,loss: 0.014,accuracy:1.00000
INFO:tensorflow:trainstep:  7100,loss: 0.009,accuracy:1.00000
INFO:tensorflow:trainstep:  7120,loss: 0.017,accuracy:0.99000
INFO:tensorflow:trainstep:  7140,loss: 0.020,accuracy:1.00000
INFO:tensorflow:trainstep:  7160,loss: 0.005,accuracy:1.00000
INFO:tensorflow:trainstep:  7180,loss: 0.009,accuracy:1.00000
INFO:tensorflow:teststep:  7200,loss: 0.012,accuracy:1.00000
INFO:tensorflow:trainstep:  7200,loss: 0.012,accuracy:1.00000
INFO:tensorflow:trainstep:  7220,loss: 0.014,accuracy:1.00000
INFO:tensorflow:trainstep:  7240,loss: 0.003,accuracy:1.00000
INFO:tensorflow:trainstep:  7260,loss: 0.015,accuracy:0.99000
INFO:tensorflow:trainstep:  7280,loss: 0.008,accuracy:1.00000
INFO:tensorflow:trainstep:  7300,loss: 0.007,accuracy:1.00000
INFO:tensorflow:trainstep:  7320,loss: 0.004,accuracy:1.00000
INFO:tensorflow:trainstep:  7340,loss: 0.024,accuracy:0.99000
INFO:tens

INFO:tensorflow:trainstep:  9420,loss: 0.010,accuracy:1.00000
INFO:tensorflow:trainstep:  9440,loss: 0.009,accuracy:1.00000
INFO:tensorflow:teststep:  9450,loss: 0.004,accuracy:1.00000
INFO:tensorflow:trainstep:  9460,loss: 0.010,accuracy:1.00000
INFO:tensorflow:trainstep:  9480,loss: 0.008,accuracy:1.00000
INFO:tensorflow:trainstep:  9500,loss: 0.082,accuracy:0.99000
INFO:tensorflow:trainstep:  9520,loss: 0.006,accuracy:1.00000
INFO:tensorflow:trainstep:  9540,loss: 0.002,accuracy:1.00000
INFO:tensorflow:trainstep:  9560,loss: 0.005,accuracy:1.00000
INFO:tensorflow:trainstep:  9580,loss: 0.004,accuracy:1.00000
INFO:tensorflow:teststep:  9600,loss: 0.006,accuracy:1.00000
INFO:tensorflow:trainstep:  9600,loss: 0.006,accuracy:1.00000
INFO:tensorflow:trainstep:  9620,loss: 0.015,accuracy:1.00000
INFO:tensorflow:trainstep:  9640,loss: 0.007,accuracy:1.00000
INFO:tensorflow:trainstep:  9660,loss: 0.004,accuracy:1.00000
INFO:tensorflow:trainstep:  9680,loss: 0.002,accuracy:1.00000
INFO:tenso